In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from datetime import datetime



## Load Task Duration Config
For each task type that could be generated we need an average duration and a maximum duration.

In [2]:
#Load task duration dataframe
#task_duration_df = pd.read_csv('task_duration_config.csv')
#task_duration_df.set_index('Task', inplace=True)

data = [['New Client Onboarding Request', 0.5,1.5], 
        ['Review Documents', 0.25,  2.5], 
        ['Automated Scoreboarding', 0.1, 0.15],
        ['Manual Scoreboarding', 1.0, 3.0],
        ['Update Backend Systems', 0.25, 0.5],
        ['Notification Review Request Completed', 0.1, 0.15]
       ]

task_duration_df = pd.DataFrame(data, columns=['Task', 'Avg', 'Max'])
task_duration_df.set_index('Task', inplace=True)

In [3]:
task_duration_df.head(10)

,Avg,Max
Task,,
New Client Onboarding Request,0.50,1.50
Review Documents,0.25,2.50
Automated Scoreboarding,0.10,0.15
Manual Scoreboarding,1.00,3.00
Update Backend Systems,0.25,0.50
Notification Review Request Completed,0.10,0.15


## Utility Functions

### Generate Process Instances
This function manages the creation of process instances for a specific process variant. A process variant is simply the list of tasks in execution order, including any loops. For example ['Wake Up', 'Breakfast', 'Work', 'Dinner' 'Sleep']

In [4]:
def generate_process_instances(process_variant, qty):

    global start_date_time
    global instance_counter
    task_list = []
    for x in range(0, qty):
        tasks = build_task_list(instance_counter, process_variant, start_date_time, task_duration_df)
        for task in tasks:
            task_list.append(task)

        # Increment the start time by 24 hour
        start_date_time = start_date_time + timedelta(hours=24)
        # Increment the process_id
        instance_counter = instance_counter +1
        
    return task_list

## Build Task List
The build_task_list function creates a list of tasks for a specific instance of a process. The sequence of tasks is defined in the variable called process_variant. Random task durations are calculated using the task_durations_df that contains the task name, the average duration and the max duration.  

This code generates a random duration by leveraging the numpy lognormal function that gives a random
number drawn from a log normal distribution. Look up a picture of log normal distributions and you'll see 
why this is useful for generating random durations based on a mean and std deviation.
most samples are near the average with a long tail stretching towards infinity.


In [5]:
def build_task_list(instance_id, process_variant, start_date_time, task_duration_df):
    instance_task_list = []
    rnd = np.random.default_rng()  # a random number generator

    # process_variant is a series
    for task_type in process_variant:
        # get the avg and max durations from the task_df dataframe using task_type as the key
        avg_dur = task_duration_df.loc[task_type, 'Avg']
        max_dur = task_duration_df.loc[task_type, 'Max']

        sigma = (max_dur - avg_dur) / max_dur  # std dev
        log_mean = np.log(avg_dur)  # can't pass the mean duration into lognormal until it has been logged itself 
        delta = rnd.lognormal(log_mean, sigma)  # get a random sample from a log normal distribution with a std dev
        end_time = start_date_time + timedelta(hours=delta)
        task = [instance_id, task_type, start_date_time, end_time]
        instance_task_list.append(task)

        # set the start time of the next tasks to be 5 mins after the end time of this task
        start_date_time = end_time + timedelta(minutes=5)

    return instance_task_list

## Delay Task Start Time
This function shifts the task start time for a specific task with a specific attribute set to a specific value. For example, to delay the start time of breakfast by 1hour "task_list, User, Gerry, Eat Breakfast, 1"

This function will apply the same delay to every subsequent task of the process instance. 

In [6]:
def delayStartTime(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1

    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']


        # Have we found the target activity
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process

        # Are we still processing the same process that we were when we found the target activity ?
        # If so we are moving all subsequent tasks back by the value of the timeshift parameter,
        # this means shifting the start time and the end time
        if current_process == process_being_modified:
            # shift the start time
            current_start = row['start_time']
            shifted_start = current_start + timedelta(hours=timeShift)
            tasks.loc[x, 'start_time'] = shifted_start
            
            # shift the end time
            current_end = row['end_time']
            shifted_end = current_end + timedelta(hours=timeShift)
            tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

## Increase Duration
This will increase the duration of the target task for each process and alter the timings of any subsequent tasks
When iterating over tasks there are four potential conditions:
1. We haven't found the target task in the tasks of a process - do nothing.
2. We have found the target task so just increase the end time
3. We have found a task the follows the target task, increase the start time and end time by the timedelta

This logic treats repeat target tasks just like any other task that follows the first target, it only delays the first encounter with 
the target task. 

In [7]:
def increaseDuration(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1
    target_task_counter = 0
    
    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']
        processing_target_task = False
        
        # reset the target task counter when we come across a new process
        if current_process !=  process_being_modified:
            target_task_counter = 0

        # Have we found a task of the target type with a target attribute matching the target value?
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process
            target_task_counter = target_task_counter + 1
            
            # If this is the first target task in this process then
            # just increase the duration by increasing the end_time
            if target_task_counter == 1: 
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            else:
                current_start = row['start_time']
                shifted_start = current_start + timedelta(hours=timeShift)
                tasks.loc[x, 'start_time'] = shifted_start
            
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
                
        # If we are processing ANY subsequent task that follows the target task 
        # this means shifting the start time and the end time.
        elif target_task_counter > 0:
                current_start = row['start_time']
                shifted_start = current_start + timedelta(hours=timeShift)
                tasks.loc[x, 'start_time'] = shifted_start
            
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

## Initialise Data Generator

In [8]:
start_date_time = datetime(2017, 11, 28, 18, 00, 00)
instance_time_offset = 24
instance_counter = 1


## Create Happy Path Instances

In [9]:

qty = 900
happy_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

happy_path_task_list = []
happy_path_task_list = generate_process_instances(happy_path, qty)
happy_path_task_list_df = pd.DataFrame(happy_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [10]:
happy_path_task_list_df.head()

,0,1,2,3
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480


## Create Instances Requiring Manual Scoreboarding

In [11]:

qty = 80
manual_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

manual_path_task_list = []
manual_path_task_list = generate_process_instances(manual_path, qty)
manual_path_task_list_df = pd.DataFrame(manual_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [12]:
manual_path_task_list_df.head()

,0,1,2,3
0,901,New Client Onboarding Request,2020-05-17 18:00:00.000000,2020-05-17 18:39:44.664783
1,901,Review Documents,2020-05-17 18:44:44.664783,2020-05-17 19:12:01.179872
2,901,Automated Scoreboarding,2020-05-17 19:17:01.179872,2020-05-17 19:23:38.866205
3,901,Manual Scoreboarding,2020-05-17 19:28:38.866205,2020-05-17 20:44:13.742581
4,901,Update Backend Systems,2020-05-17 20:49:13.742581,2020-05-17 21:00:40.366571


## Create Instance that loop
Create processes that loop back to Review Docs from Manual Scoreboarding.

In [13]:
qty = 20
loop_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Review Documents",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

loop_path_task_list = []
loop_path_task_list = generate_process_instances(loop_path, qty)
loop_path_task_list_df = pd.DataFrame(loop_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [14]:
# Can only concat two dataframes at a time
df = pd.concat([happy_path_task_list_df,manual_path_task_list_df], axis=0)
df = pd.concat([df,loop_path_task_list_df], axis=0)

In [15]:
df.columns = ['process_id', 'task', 'start_time', 'end_time']

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 139
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   process_id  5120 non-null   int64         
 1   task        5120 non-null   object        
 2   start_time  5120 non-null   datetime64[ns]
 3   end_time    5120 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 200.0+ KB


In [17]:
# Confirm how many processes
len(df["process_id"].unique())

1000

In [18]:
df.head(10)

,process_id,task,start_time,end_time
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:11:13.263380
6,2,Review Documents,2017-11-29 18:16:13.263380,2017-11-29 18:24:50.850333
7,2,Automated Scoreboarding,2017-11-29 18:29:50.850333,2017-11-29 18:37:19.760233
8,2,Update Backend Systems,2017-11-29 18:42:19.760233,2017-11-29 19:00:57.035926
9,2,Notification Review Request Completed,2017-11-29 19:05:57.035926,2017-11-29 19:10:15.819603


## Add Task Level Business Data : User

In [19]:

# Add a column for User
df["user"] = ""

In [20]:
def setRandomUser(row):
    match row["task"]:
        case "Review Documents":
            return random.choice(['Rod','Jane','Freddy'])
        case "New Client Onboarding Request":
            return random.choice(['Clive','Francis','Nick','Seb','Tom'])
        case "Manual Scoreboarding":
            return random.choice(['Sharon','Susan', 'Sam'])
        case "Update Backend Systems":
            return "RPA"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return row["user"]

In [21]:
df["user"] = df.apply(setRandomUser, axis=1)

In [22]:
df.head(20)

,process_id,task,start_time,end_time,user
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541,Francis
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159,Jane
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968,SYSTEM
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766,RPA
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:11:13.263380,Francis
6,2,Review Documents,2017-11-29 18:16:13.263380,2017-11-29 18:24:50.850333,Jane
7,2,Automated Scoreboarding,2017-11-29 18:29:50.850333,2017-11-29 18:37:19.760233,SYSTEM
8,2,Update Backend Systems,2017-11-29 18:42:19.760233,2017-11-29 19:00:57.035926,RPA
9,2,Notification Review Request Completed,2017-11-29 19:05:57.035926,2017-11-29 19:10:15.819603,SYSTEM


## Add Task Level Business Data : UserGroup

In [23]:
# Add a column for UserGroup
df["user_group"] = ""

In [24]:
def setUserGroup(row):
    match row["task"]:
        case "Review Documents":
            return "Operations"
        case "New Client Onboarding Request":
            return "Sales" 
        case "Manual Scoreboarding":
            return "Risk"
        case "Update Backend Systems":
            return "SYSTEM"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return ""

In [25]:
df["user_group"] = df.apply(setUserGroup, axis=1)

In [26]:
df.head(20)

,process_id,task,start_time,end_time,user,user_group
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541,Francis,Sales
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159,Jane,Operations
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968,SYSTEM,SYSTEM
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766,RPA,SYSTEM
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480,SYSTEM,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:11:13.263380,Francis,Sales
6,2,Review Documents,2017-11-29 18:16:13.263380,2017-11-29 18:24:50.850333,Jane,Operations
7,2,Automated Scoreboarding,2017-11-29 18:29:50.850333,2017-11-29 18:37:19.760233,SYSTEM,SYSTEM
8,2,Update Backend Systems,2017-11-29 18:42:19.760233,2017-11-29 19:00:57.035926,RPA,SYSTEM
9,2,Notification Review Request Completed,2017-11-29 19:05:57.035926,2017-11-29 19:10:15.819603,SYSTEM,SYSTEM


## Add Process Instance Business Data : Industry
Industy won't change during the process so all tasks for a given process ID must have the same value

In [27]:
df["industry"] = ""

In [28]:
process_id_list = df["process_id"].unique()

for process_id in process_id_list:
    industry = random.choice(['Federal','Finance','Healthcare','Insurance','Telecom'])
    df.loc[df["process_id"].eq(process_id), "industry"] = industry
    
                          
    

In [29]:
df.head(20)

,process_id,task,start_time,end_time,user,user_group,industry
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541,Francis,Sales,Finance
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159,Jane,Operations,Finance
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968,SYSTEM,SYSTEM,Finance
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766,RPA,SYSTEM,Finance
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480,SYSTEM,SYSTEM,Finance
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:11:13.263380,Francis,Sales,Healthcare
6,2,Review Documents,2017-11-29 18:16:13.263380,2017-11-29 18:24:50.850333,Jane,Operations,Healthcare
7,2,Automated Scoreboarding,2017-11-29 18:29:50.850333,2017-11-29 18:37:19.760233,SYSTEM,SYSTEM,Healthcare
8,2,Update Backend Systems,2017-11-29 18:42:19.760233,2017-11-29 19:00:57.035926,RPA,SYSTEM,Healthcare
9,2,Notification Review Request Completed,2017-11-29 19:05:57.035926,2017-11-29 19:10:15.819603,SYSTEM,SYSTEM,Healthcare


## Add Process Instance Business Data : Service Charge
Service charge will be a random choice based on industry.

In [30]:
df["service_charge"] = 0

In [31]:
def getServiceChargeByIndustry(industry):
    match industry:
        case "Federal":
            return random.choice([3000, 6000, 8000])
        case "Finance":
            return random.choice([10000, 12000, 20000])
        case "Healthcare":
            return random.choice([15000, 20000, 25000])
        case "Insurance":
            return 45000
        case "Telecom":
            return 49000
        case _:
            return 64000

In [32]:
last_process = -1
service_charge = 0

process_id_list = df["process_id"].unique()

for process_id in process_id_list:
        # get the tasks for this process instance, use copy() to make it clear to the interpreter we know
        # this is a copy to supress any warnings about updating copies
        process_instance_tasks = df.loc[df.process_id == process_id].copy()
        # Get the fir row out of the task list and get the industry value
        industry = process_instance_tasks.iloc[0]['industry']
        
        # call function to get a semi-random service charge for the industry
        service_charge = getServiceChargeByIndustry(industry)
        
        # set the service charge in every row for this process instance
        df.loc[df.process_id == process_id, 'service_charge'] = service_charge
    


In [33]:
df.head(50)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:24:31.705541,Francis,Sales,Finance,12000
1,1,Review Documents,2017-11-28 18:29:31.705541,2017-11-28 19:05:19.125159,Jane,Operations,Finance,12000
2,1,Automated Scoreboarding,2017-11-28 19:10:19.125159,2017-11-28 19:15:37.475968,SYSTEM,SYSTEM,Finance,12000
3,1,Update Backend Systems,2017-11-28 19:20:37.475968,2017-11-28 19:32:25.774766,RPA,SYSTEM,Finance,12000
4,1,Notification Review Request Completed,2017-11-28 19:37:25.774766,2017-11-28 19:44:15.431480,SYSTEM,SYSTEM,Finance,12000
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:11:13.263380,Francis,Sales,Healthcare,20000
6,2,Review Documents,2017-11-29 18:16:13.263380,2017-11-29 18:24:50.850333,Jane,Operations,Healthcare,20000
7,2,Automated Scoreboarding,2017-11-29 18:29:50.850333,2017-11-29 18:37:19.760233,SYSTEM,SYSTEM,Healthcare,20000
8,2,Update Backend Systems,2017-11-29 18:42:19.760233,2017-11-29 19:00:57.035926,RPA,SYSTEM,Healthcare,20000
9,2,Notification Review Request Completed,2017-11-29 19:05:57.035926,2017-11-29 19:10:15.819603,SYSTEM,SYSTEM,Healthcare,20000


# Stories
Make a copy of the dataframe so we can restart here if required.

In [34]:
story_df = df.copy()

## Story : 1 - Rod is Too Busy
Rod is always very busy doing other work, so he may not be able to start right away.
Lets take a look at two `Review Documents` tasks that have been done by Rod. 

In [35]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
21,5,Review Documents,2017-12-02 18:37:28.058347,2017-12-02 18:42:35.317775,Rod,Operations,Federal,6000
26,6,Review Documents,2017-12-03 18:44:52.131291,2017-12-03 18:49:40.004211,Rod,Operations,Healthcare,15000


Delay the start time of Rod's `Review Documents` tasks by 1 hour

In [36]:
# If user is Rod and activity is `Review Documents` shift the start_time by 1 hour
# 
story_df = delayStartTime(story_df, 'user','Rod', 'Review Documents', 1)

Lets see the result of this change:

In [37]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
21,5,Review Documents,2017-12-02 19:37:28.058347,2017-12-02 19:42:35.317775,Rod,Operations,Federal,6000
26,6,Review Documents,2017-12-03 19:44:52.131291,2017-12-03 19:49:40.004211,Rod,Operations,Healthcare,15000


## Story : 2 - Complex Doc Review
Increase the time taken for doc review by 2hrs when the industry is federal. 
The rationale here is that the documentation requirements are greater.

Lets look at some `Review Documents` where the industry is `Federal`

In [38]:
filter = (story_df['task'] == 'Review Documents') & (story_df['industry'] == 'Federal')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
16,4,Review Documents,2017-12-01 18:48:44.178429,2017-12-01 19:07:34.895552,Jane,Operations,Federal,6000
21,5,Review Documents,2017-12-02 19:37:28.058347,2017-12-02 19:42:35.317775,Rod,Operations,Federal,6000


In [39]:
story_df[story_df['process_id'] == 4]

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
15,4,New Client Onboarding Request,2017-12-01 18:00:00.000000,2017-12-01 18:43:44.178429,Nick,Sales,Federal,6000
16,4,Review Documents,2017-12-01 18:48:44.178429,2017-12-01 19:07:34.895552,Jane,Operations,Federal,6000
17,4,Automated Scoreboarding,2017-12-01 19:12:34.895552,2017-12-01 19:20:49.628808,SYSTEM,SYSTEM,Federal,6000
18,4,Update Backend Systems,2017-12-01 19:25:49.628808,2017-12-01 19:32:20.893086,RPA,SYSTEM,Federal,6000
19,4,Notification Review Request Completed,2017-12-01 19:37:20.893086,2017-12-01 19:45:14.830472,SYSTEM,SYSTEM,Federal,6000


In [40]:
story_df = increaseDuration(story_df, 'industry','Federal', 'Review Documents', 2)

In [41]:
story_df[story_df['process_id'] == 4]

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
15,4,New Client Onboarding Request,2017-12-01 18:00:00.000000,2017-12-01 18:43:44.178429,Nick,Sales,Federal,6000
16,4,Review Documents,2017-12-01 18:48:44.178429,2017-12-01 21:07:34.895552,Jane,Operations,Federal,6000
17,4,Automated Scoreboarding,2017-12-01 21:12:34.895552,2017-12-01 21:20:49.628808,SYSTEM,SYSTEM,Federal,6000
18,4,Update Backend Systems,2017-12-01 21:25:49.628808,2017-12-01 21:32:20.893086,RPA,SYSTEM,Federal,6000
19,4,Notification Review Request Completed,2017-12-01 21:37:20.893086,2017-12-01 21:45:14.830472,SYSTEM,SYSTEM,Federal,6000


## Export Finished Dataset

In [42]:
story_df.to_csv('pm_tasks_1k.csv', index=False, date_format='%d-%m-%Y %H:%M:%S')

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
df[df['process_id'] == 989]

In [ ]:
story_df = df.copy()

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
story_df = delayStartTime(story_df, 'user','Rod', 'Review Documents', 1)

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
story_df = increaseDuration(story_df, 'industry','Federal', 'Review Documents', 2)

In [ ]:
story_df[story_df['process_id'] == 989]

## Export Finished Dataset

In [ ]:
story_df.to_csv('pm_tasks_1k.csv', index=False, date_format='%d-%m-%Y %H:%M:%S')

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
df[df['process_id'] == 989]

In [ ]:
story_df = df.copy()

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
story_df = delayStartTime(story_df, 'user','Rod', 'Review Documents', 1)

In [ ]:
story_df[story_df['process_id'] == 989]

In [ ]:
story_df = increaseDuration(story_df, 'industry','Federal', 'Review Documents', 2)

In [ ]:
story_df[story_df['process_id'] == 989]